In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from compmec import nurbs
from helper import getH, getD

# Formulação FEM para Navier-Stokes

No notebook anterior tratamos EDPs de diferentes tipos, todos lineares. Mas a mema abordagem utilizada não funciona para o problema que trataremos aqui: resolver a Equação de Navier Stokes.

Antes disso, teremos as quantidades:

* Velocidade do fluido: $\mathbf{u} = \left(u_1, \ u_2, \ u_3\right) = \left(u, \ v, \ w\right)$
* Pressão do fluido: $p$
* Fator de campo: $\mathbf{g}$
* Densidade: $\rho$
* Fluido newtoniano de viscosidade: $\mu$
* Tempo: $t$
* Espaço: $\left(x, \ y, \ z\right) = \left(x_1, \ x_2, \ x_3\right)$

Resolver a equação de Navier Stokes se resume a encontrar os valores da velocidade $\mathbf{u}$ e pressão $p$ ao longo do domínio $\Omega_{xyzt}$.

Na notação vetorial temos a equação de balanço de massa dado por $(1.1)$ e o balanço de momento dado por $(1.2)$

$$\dfrac{\partial \rho}{\partial t} + \nabla \cdot \left(\rho \mathbf{u}\right) = 0 \tag{1.1}$$

$$\dfrac{\partial \left(\rho \mathbf{u}\right)}{\partial t} + \nabla \cdot \left( \rho \mathbf{u} \otimes \mathbf{u} \right) = -\nabla p + \rho \mathbf{g} + \mu \nabla^2 \mathbf{u}\tag{1.2}$$


Na notação indicial, $(1.1)$ e $(1.2)$ se tornam $(1.3)$ e $(1.4)$

$$\dfrac{\partial \rho}{\partial t} + \sum_{i = 1}^{3} \dfrac{\partial \left(\rho u_i\right)}{\partial x_i} = 0
\tag{1.3}$$
$$\dfrac{\partial \rho u_i}{\partial t} + \sum_{j=1}^{3} \dfrac{\partial \left(\rho u_i u_j\right)}{\partial x_j} = -\dfrac{\partial p}{\partial x_i} + \rho g_i + \mu \sum_{j=1}^{3} \dfrac{\partial^2 u_i}{\partial x_j^2} \ \ \ \ \ \forall i \tag{1.4}$$

Para o primeiro exemplo, simplificaremos: Utilizaremos uma malha bidimensional $u_3 = 0$ de um fluido incompressível $\rho = \text{const}$, com campo nulo $\mathbf{g} = \mathbf{0}$.

Também fazemos a adimensionalização do problema, resultando em $(1.5)$, $(1.6)$ e $(1.7)$:

$$
\begin{align*}
\dfrac{\partial u}{\partial x} + \dfrac{\partial v}{\partial y} & = 0 \tag{1.5} \\ \dfrac{\partial u}{\partial t} + u \dfrac{\partial u}{\partial x} + v\dfrac{\partial u}{\partial y} & = -\dfrac{\partial p}{\partial x} + \mu \left(\dfrac{\partial^2 u}{\partial x^2}+\dfrac{\partial^2 u}{\partial y^2}\right) \tag{1.6} \\
\dfrac{\partial v}{\partial t} + u\dfrac{\partial v}{\partial x} + v\dfrac{\partial v}{\partial y} & = -\dfrac{\partial p}{\partial y} + \mu \left(\dfrac{\partial^2 v}{\partial x^2}+\dfrac{\partial^2 v}{\partial y^2}\right) \tag{1.7}
\end{align*}
$$

Na malha $\Omega_{xyt} = \left[0, \ 1\right] \times \left[0, \ 1\right] \times \left[0, \ \infty\right)$.

### FEM com BSpline

Com a equação de navier stokes, iremos utilizar BSpline apenas para as variáveis no espaço.
Desta forma, calculamos progressivamente no tempo, resolvendo um sistema para cada passo de tempo e obtendo uma EDO em relação a $t$ .

Então, supondo que conhecemos as funções no tempo $t_{l} \in \left[0, \ \infty\right)$

$$\begin{align*}u_{l}(x, \ y) & = \sum_{i=0}^{nx-1} \sum_{j=0}^{ny-1} N_{i,px}(x) \cdot N_{j, py}(y) \cdot U_{l,i,j} \tag{1.8} \\
v_{l}(x, \ y) & = \sum_{i=0}^{nx-1} \sum_{j=0}^{ny-1} N_{i,px}(x) \cdot N_{j, py}(y) \cdot V_{l,i,j} \tag{1.9} \\ p_{l}(x, \ y) & = \sum_{i=0}^{nx-1} \sum_{j=0}^{ny-1} N_{i,px}(x) \cdot N_{j, py}(y) \cdot P_{l,i,j} \tag{1.10}\end{align*}$$

Utilizamos $(1.5)$, $(1.6)$ e $(1.7)$ para encontrar as funções $u_{l+1}(x, \ y)$, $v_{l+1}(x, \ y)$ e $p_{l+1}(x, \ y)$, totalizando $3n_xn_y$ variáveis ($\left[U_{l+1}\right]$, $\left[V_{l+1}\right]$ e $\left[P_{l+1}\right]$) a serem encontradas:

$$\left[\ \bigcirc \ \right] \cdot \left[U_{l+1}\right] + \left[\ \bigcirc \ \right]  \cdot \left[V_{l+1}\right]  = \left[0\right]$$
$$\left[\ \bigcirc \ \right] \cdot \left[U_{l+1}\right] +  \left[ \ \bigcirc \ \right] \cdot \left[P_{l+1}\right] = \left[\ \bigcirc \ \right]$$
$$\left[\ \bigcirc \ \right] \cdot \left[V_{l+1}\right] +  \left[ \ \bigcirc \ \right] \cdot \left[P_{l+1}\right] = \left[\ \bigcirc \ \right]$$

Contudo, utilizaremos o método das projeções:

1. 

A partir daqui omitiremos o indice $l$.

### Formulação de Galerkin

Na formulação de Galerkin, teremos para a Equação $(1.5)$:

$$\int_{\Omega} \left(\dfrac{\partial u}{\partial x} + \dfrac{\partial u}{\partial y}\right)\cdot \varphi \ d\Omega = 0 \ \ \ \ \ \forall \varphi$$
$$\begin{align*}\sum_{a=0}^{nx-1} \sum_{b=0}^{ny-1} & \left(\int_{0}^{1} N_{i,px} \dfrac{dN_{a,px}}{dx} \ dx\right)\left(\int_{0}^{1}N_{j,py}N_{b,py} \ dy \right)U_{a,b} \\ & +\left(\int_{0}^{1} N_{i,px} N_{a,px} \ dx\right)\left(\int_{0}^{1}N_{j,py}\dfrac{dN_{b,py}}{dy} \ dy \right)V_{a,b} = 0 \ \ \ \ \forall i, \ j \end{align*} $$
$$\sum_{a,b}\left(\left[H_{px, px-1}\right] \left[D_{px}\right]^{T}\right)_{ia}\left[H_{py,py}\right]_{jb} \left[U\right]_{ab} + \left[H_{px,px}\right]_{ia}\left(\left[H_{py, py-1}\right] \left[D_{py}\right]^{T}\right)_{jb} \left[V\right]_{ab}= 0 \ \ \ \forall i, \ j$$

Já para a Equação $(1.6)$ teremos

$$\int_{\Omega} \left( \dfrac{\partial u}{\partial t} + u\dfrac{\partial u}{\partial x} + v\dfrac{\partial u}{\partial y} + \dfrac{\partial p}{\partial x} - \mu \left(\dfrac{\partial^2 u}{\partial x^2}+\dfrac{\partial^2 u}{\partial y^2}\right)\right) N_{i,px}N_{j,py} \ d\Omega = 0 \ \ \ \ \forall i, \ j$$
$$\begin{align*} \sum_{a, b} & \left[H_{px,px}\right]_{i,a} \left[H_{py,py}\right]_{j,b} \dfrac{dU_{a,b}}{dt} \\
+ \sum_{a1, b1, a2, b2} & \left(\left[H_{px,px,px-1}\right]\left[D_{px}^{T}\right]\right)_{i,a1,a2} \left[H_{py,py,py}\right]_{j,b1,b2} U_{a1,b1} U_{a2,b2} \\
+ \sum_{a_1, b_1, a_2, b_2} & \left[H_{px,px,px}\right]_{i,a1,a2} \left(\left[H_{py,py,py-1}\right]\left[D_{py}\right]^{T}\right)_{j,b1,b2} V_{a1,b1}U_{a2,b2} \\
+ \sum_{a, b}  & \left(\left[H_{px,px-1}\right]\left[D_{px}\right]^{T}\right)_{i,a} \left[H_{py,py}\right]_{j,b} P_{a,b} \\ 
- \mu \sum_{a,b} &  \left(\left[X_{bound}\right]\left[D_{px}\right]^{T}-\left[D_{px}\right]\left[H_{px-1,px-1}\right]\left[D_{px}\right]^{T}\right)_{i,a}\left[H_{py,py}\right]_{j,b} U_{a,b} \\ 
- \mu \sum_{a,b}& \left[H_{px,px}\right]_{i,a} \left(\left[Y_{bound}\right]\left[D_{py}\right]^{T}-\left[D_{py}\right]\left[H_{py-1,py-1}\right]\left[D_{py}\right]^{T}\right)_{j,b} U_{a,b} = 0 \ \ \ \forall i, \ j\end{align*}$$

Da mesma forma com a equação $(1.7)$ obtemos:


$$\begin{align*} \sum_{a, b} & \left[H_{px,px}\right]_{i,a} \left[H_{py,py}\right]_{j,b} \dfrac{dV_{a,b}}{dt} \\
+ \sum_{a1, b1, a2, b2} & \left(\left[H_{px,px,px-1}\right]\left[D_{px}^{T}\right]\right)_{i,a1,a2} \left[H_{py,py,py}\right]_{j,b1,b2} U_{a1,b1} U_{a2,b2} \\
+ \sum_{a_1, b_1, a_2, b_2} & \left[H_{px,px,px}\right]_{i,a1,a2} \left(\left[H_{py,py,py-1}\right]\left[D_{py}\right]^{T}\right)_{j,b1,b2} V_{a1,b1}U_{a2,b2} \\
+ \sum_{a, b}  & \left(\left[H_{px,px-1}\right]\left[D_{px}\right]^{T}\right)_{i,a} \left[H_{py,py}\right]_{j,b} P_{a,b} \\ 
- \mu \sum_{a,b} &  \left(\left[X_{bound}\right]\left[D_{px}\right]^{T}-\left[D_{px}\right]\left[H_{px-1,px-1}\right]\left[D_{px}\right]^{T}\right)_{i,a}\left[H_{py,py}\right]_{j,b} V_{a,b} \\ 
- \mu \sum_{a,b}& \left[H_{px,px}\right]_{i,a} \left(\left[Y_{bound}\right]\left[D_{py}\right]^{T}-\left[D_{py}\right]\left[H_{py-1,py-1}\right]\left[D_{py}\right]^{T}\right)_{j,b} V_{a,b} = 0 \ \ \ \forall i, \ j\end{align*}$$

In [ ]:
@np.vectorize
def initial_conditions_u(x: float, y: float) -> float:
    offset = 0.99
    if y <= offset:
        return 0
    return ((y-offset)/(1-offset))**2 * np.sin(np.pi*x)

@np.vectorize
def initial_conditions_v(x: float, y: float) -> float:
    return 0

@np.vectorize
def initial_conditions_p(x: float, y: float) -> float:
    return 0

px, py = 2, 2
nx, ny = 3, 3
Ux = nurbs.GeneratorKnotVector.uniform(px, nx)
Uy = nurbs.GeneratorKnotVector.uniform(py, ny)
Nx = nurbs.SplineBaseFunction(Ux)
Ny = nurbs.SplineBaseFunction(Uy)

Xbound = np.tensordot(Nx[:, px](1), Nx[:, px-1](0), axes=0)
Xbound -= np.tensordot(Nx[:, px](0), Nx[:, px-1](0), axes=0)
Ybound = np.tensordot(Ny[:, py](1), Ny[:, py-1](0), axes=0)
Ybound -= np.tensordot(Ny[:, py](0), Ny[:, py-1](0), axes=0)
Hpxpx = getH(Nx, px, px)
Hpypy = getH(Ny, py, py)
Hpx1px1 = getH(Nx, px-1, px-1)
Hpy1py1 = getH(Ny, py-1, py-1)
Hpxpxpx = getH(Nx, px, px, px)
Hpypypy = getH(Ny, py, py, py)
Hpxpxpx1 = getH(Nx, px, px, px-1)
Hpypypy1 = getH(Ny, py, py, py-1)
Dpx = getD(px, Ux)
Dpy = getD(px, Uy)

Termx = Xbound @ Dpx.T - Dpx @ Hpx1px1 @ Dpx.T
Termy = Ybound @ Dpy.T - Dpy @ Hpy1py1 @ Dpy.T
Mat1 = np.tensordot(Termx, Hpypy, axes=0)
Mat1 += np.tensordot(Hpxpx, Termy, axes=0)
Mat2 = np.tensordot(Hpxpxpx1 @ Dpx.T, Hpypypy, axes=0)
Mat2 += np.tensordot(Hpxpxpx, Hpypypy1 @ Dpy.T, axes=0)


